[pathpy](https://www.pathpy.net/index.html)

[documentation](https://www.pathpy.net/manual/index.html)

In [1]:
import pathpy as pp
import networkx as nx
import numpy as np
import pandas as pd

In [2]:
n = pp.Network()
n.add_edge('a', 'b')
n.add_edge('b', 'c')
print(n)

Undirected network
Nodes:				3
Links:				2



### Basics

In [3]:
n = pp.Network(directed=True)
n.add_edge('a', 'c')
n.add_edge('b', 'c')
n.add_edge('c', 'd')
n.add_edge('c', 'e')
print(n)

Directed network
Nodes:				5
Links:				4



In [4]:
# for print visualisation in jupyter notebook
n

In [5]:
# or the same
pp.visualisation.plot(n)

In [6]:
# centralities
c = pp.algorithms.centralities.betweenness(n)
print(c)

2021-01-26 20:33:37 [Severity.INFO]	Calculating betweenness centralities ...
defaultdict(<function betweenness.<locals>.<lambda> at 0x7f159f478f70>, {'c': 4.0, 'a': 0, 'b': 0, 'd': 0, 'e': 0})


In [7]:
# sentralities usage for visualisation
style = {}
style['node_size'] = {v:7+u for v,u in c.items()}
pp.visualisation.plot(n, **style)

### Temporal networks

In [8]:
t = pp.TemporalNetwork()
t.add_edge('a', 'b', 1)
t.add_edge('b', 'a', 3)
t.add_edge('b', 'c', 3)
t.add_edge('d', 'c', 4)
t.add_edge('c', 'd', 5)
t.add_edge('c', 'b', 6)
print(t)

Nodes:			4
Time-stamped links:	6
Links/Nodes:		1.5
Observation period:	[1, 6]
Observation length:	 5 
Time stamps:		 5 
Avg. inter-event dt:	 1.25
Min/Max inter-event dt:	 1/2


In [9]:
pp.visualisation.plot(t)

In [10]:
style = {    
  'ts_per_frame': 1, 
  'ms_per_frame': 2000,
  'look_ahead': 2, 
  'look_behind': 2, 
  'node_size': 15, 
  'inactive_edge_width': 2,
  'active_edge_width': 4, 
  'label_color' : '#ffffff',
  'label_size' : '24px',
  'label_offset': [0,5]
  }

In [11]:
pp.visualisation.plot(t, **style)

In [12]:
# expotr to html
pp.visualisation.export_html(t, 'temporal_network.html_example', **style)

### Path statistics

[source](https://www.pathpy.net/tutorial/paths.html)

In [13]:
# instance of class Path() store path statictics for graph
p = pp.Paths()

We now have an empty Paths instance to which we can add path statistics using the method add_path(). As the first parameter, the method accepts any iterable (list, tuple, etc.) of string variables (or objects that can be cast to string). Each entry in the iterable is interpreted as one step (i.e. node) on a path through a network. The optional parameter frequency counts the number of times a path has been observed.

In [14]:
p.add_path(('a', 'c', 'd'), frequency=10)

In [15]:
print(p)

Total path count: 		10.0 
[Unique / Sub paths / Total]: 	[1.0 / 50.0 / 60.0]
Nodes:				3 
Edges:				2
Max. path length:		2
Avg path length:		2.0 
Paths of length k = 0		0.0 [ 0.0 / 30.0 / 30.0 ]
Paths of length k = 1		0.0 [ 0.0 / 20.0 / 20.0 ]
Paths of length k = 2		10.0 [ 1.0 / 0.0 / 10.0 ]



Наш примерный экземпляр содержит 10 наблюдаемых путей. Эти пути подразумевают топологию сети, состоящую из трех узлов a, c и d и двух (направленных) ребер (a, c) и (c, d). И максимальная, и средняя длина пути равны двум (где длина пути учитывает количество ребер, пройденных путем). 

Чтобы понять последние три строки и вторую строку вывода, мы должны изучить внутреннюю работу pathpy. Для вывода моделей более высокого порядка и алгоритма выбора модели, которе будут обсуждаться в следующих разделах, pathpy использует всю доступную статистику пути. Чтобы подогнать, скажем, модель второго порядка к набору путей, длина которых составляет 10 или больше, она фактически вычисляет, какие пути длины два содержатся в качестве подпутей в этих наблюдениях более длинных путей. По этой причине pathpy автоматически вычисляет статистику фактических наблюдений пути, а также статистику всех подпутей, содержащихся в этих наблюдаемых путях. 

В нашем случае у нас есть 10 наблюдений единственного уникального пути a-> b-> c длиной 2. Этот путь не является дополнительным путем более длинного пути, что объясняет последнюю строку в выходных данных выше. Каждое из этих 10 наблюдений дополнительно содержит два наблюдения подпутей a-> b и b-> c, таким образом получается число 20,0 в счетчике подпути в строке, соответствующей длине пути k = 1. Наконец, каждый из путей содержит три «пути» нулевой длины, которые являются просто наблюдениями за одним узлом (т.е. нет перехода через ребро), таким образом получается счетчик подпути 30,0 в строке k = 0. Это составляет в общей сложности 50.0 подпутей плюс 10 наблюдений одного уникального (самого длинного) пути, что объясняет, таким образом, вторую строку вывода.

Apart from adding paths as a tuple, we can also add string-encoded n-grams, using the parameter separator to specify a character that separates nodes

In [16]:
p2 = pp.Paths()
p2.add_path('b-c-e',  separator='-', frequency=10)
print(p2)

Total path count: 		10.0 
[Unique / Sub paths / Total]: 	[1.0 / 50.0 / 60.0]
Nodes:				3 
Edges:				2
Max. path length:		2
Avg path length:		2.0 
Paths of length k = 0		0.0 [ 0.0 / 30.0 / 30.0 ]
Paths of length k = 1		0.0 [ 0.0 / 20.0 / 20.0 ]
Paths of length k = 2		10.0 [ 1.0 / 0.0 / 10.0 ]



In [17]:
# some rarythmetic operations with two graphs
p = p + p2
print(p)

Total path count: 		20.0 
[Unique / Sub paths / Total]: 	[2.0 / 100.0 / 120.0]
Nodes:				5 
Edges:				4
Max. path length:		2
Avg path length:		2.0 
Paths of length k = 0		0.0 [ 0.0 / 60.0 / 60.0 ]
Paths of length k = 1		0.0 [ 0.0 / 40.0 / 40.0 ]
Paths of length k = 2		20.0 [ 2.0 / 0.0 / 20.0 ]



The result is a new instance, where 20 observed paths traverse five nodes a, b, c, d, and e across four edges (a,c), (b,c), (c,d) and (c,e)

We can use an instance of Paths to generate a directed network which is comprised of all nodes and edges that are traversed by the paths. For this, we can use the class method from_paths of the class Network.


In [18]:
n = pp.Network.from_paths(p)
print(n)

Directed network
Nodes:				5
Links:				4



In [19]:
print('Edge (a,c) has weight {0}'.format(n.edges[('a', 'c')]['weight']))

Edge (a,c) has weight 10.0


The important point is that any time-ordered data - and in fact also other data on complex networks - allows us to extract paths, which can be used to detect and reason about patterns that cannot be studied in the network topology alone. 

In [31]:
t = pp.TemporalNetwork()
t.add_edge('a', 'b', 1)
t.add_edge('b', 'a', 3)
t.add_edge('b', 'c', 3)
t.add_edge('d', 'c', 4)
t.add_edge('c', 'd', 5)
t.add_edge('c', 'b', 6)
# t.add_edge('a', 'b', '2018-08-22 09:30:22')
# t.add_edge('b', 'c', '2018-08-22 09:30:25')
# t.add_edge('c', 'a', '2018-08-22 10:30:25')

In [32]:
style = {    
    'ts_per_frame': 1, 
    'ms_per_frame': 2000,
    'look_ahead': 2, 
    'look_behind': 2, 
    'node_size': 15, 
    'inactive_edge_width': 2,
    'active_edge_width': 4, 
    'label_color' : '#ffffff',
    'label_size' : '24px',
    'label_offset': [0,5]
    }

In [33]:
pp.visualisation.plot(t, **style)

Упорядочивание и синхронизация, в которых отмеченные временными метками ребра возникают во временной сети, порождают так называемые причинные или учитывающие время пути (causal or time-respecting paths). 

Вкратце, два ребра с отметкой времени (a, b, t) и (b, c, t′), возникающие в момент времени t и t', могут вносить вклад в причинный путь a-> b-> c, только если t < t', т.е. ребро (b, c) находится после (a, c). Если мы поменяем местами две отметки времени так, чтобы ребро (b, c) появилось раньше (a, b), временного пути a-> b-> c тогда бы не существовало. Это приводит к важному наблюдению: благодаря стрелке времени хронологический порядок ребер с отметками времени во временной сети решающим образом влияет на причинные пути, то есть определяет, какие узлы могут косвенно влиять друг на друга через последовательности ребер с отметками времени. 

Более того, мы часто хотим ограничить максимальную разницу во времени между последовательными ребрами, которые вносят вклад в причинный путь. Для данных о динамических социальных взаимодействиях, охватывающих несколько лет, не имеет смысла рассматривать все хронологически упорядоченные ребра как возможные причинные пути, например, для распространения информации. В конце концов, у людей ограниченная память, и поэтому мы должны рассматривать взаимодействия, которые происходят далеко друг от друга во времени, как независимые.

Мы можем формально добавить это условие, установив максимальную разницу во времени для расчета пути. То есть мы рассматриваем только два ребра (a, b, t) и (b, c, t′) как вклад в причинный путь a-> b-> c, если 0 < t'− t ≤ delta. 

Имея это определение и установив максимальную разницу во времени, мы можем вычислить статистику причинно-следственных связей в сетевых данных с отметками времени. В частности, pathpy предоставляет алгоритмы для вычисления (или оценки) статистики причинно-следственных связей на основе экземпляра TemporalNetwork. Давайте попробуем это в приведенном выше примере временной сети. Мы также будем использовать простой цикл для перебора всех найденных путей и их вывода.

In [34]:
p = pp.path_extraction.temporal_paths.paths_from_temporal_network_dag(t, delta=1)
print(p)

for l in p.paths:
    for x in p.paths[l]:
        if p.paths[l][x][1]>0:
            print('{0} -> {1}'.format(x, p.paths[l][x][1]))

2021-01-26 21:10:25 [Severity.INFO]	Constructing time-unfolded DAG ...
2021-01-26 21:10:25 [Severity.INFO]	finished.
Directed Acyclic Graph
Nodes:		10
Roots:		4
Leaves:		5
Links:		6
Acyclic:	None

2021-01-26 21:10:25 [Severity.INFO]	Generating causal trees for 4 root nodes ...
2021-01-26 21:10:25 [Severity.INFO]	finished.
Total path count: 		5.0 
[Unique / Sub paths / Total]: 	[5.0 / 13.0 / 18.0]
Nodes:				4 
Edges:				6
Max. path length:		2
Avg path length:		1.2 
Paths of length k = 0		0.0 [ 0.0 / 11.0 / 11.0 ]
Paths of length k = 1		4.0 [ 4.0 / 2.0 / 6.0 ]
Paths of length k = 2		1.0 [ 1.0 / 0.0 / 1.0 ]

('c', 'b') -> 1.0
('a', 'b') -> 1.0
('b', 'c') -> 1.0
('b', 'a') -> 1.0
('d', 'c', 'd') -> 1.0


Для delta = 1 легко проверить, что это правильно в приведенном выше примере временной сети. Есть только одна пара (направленных) ребер (d, c, 4) и (c, d, 5), которая вносит вклад в причинный путь длины два. Кроме того, у нас есть четыре ребра с отметками времени, каждое из которых представляет собой тривиальный причинный путь длины один.

Хотя относительно легко проверить статистику пути для максимальной разницы во времени delta = 1, уже для delta = 2 ситуация значительно усложняется:

In [35]:
p = pp.path_extraction.paths_from_temporal_network_dag(t, delta=2)
print(p)

for l in p.paths:
    for x in p.paths[l]:
        if p.paths[l][x][1]>0:
            print('{0} -> {1}'.format(x, p.paths[l][x][1]))

2021-01-26 21:10:28 [Severity.INFO]	Constructing time-unfolded DAG ...
2021-01-26 21:10:28 [Severity.INFO]	finished.
Directed Acyclic Graph
Nodes:		13
Roots:		2
Leaves:		8
Links:		12
Acyclic:	None

2021-01-26 21:10:28 [Severity.INFO]	Generating causal trees for 2 root nodes ...
2021-01-26 21:10:28 [Severity.INFO]	finished.
Total path count: 		4.0 
[Unique / Sub paths / Total]: 	[4.0 / 24.0 / 28.0]
Nodes:				4 
Edges:				6
Max. path length:		3
Avg path length:		2.25 
Paths of length k = 0		0.0 [ 0.0 / 13.0 / 13.0 ]
Paths of length k = 1		0.0 [ 0.0 / 9.0 / 9.0 ]
Paths of length k = 2		3.0 [ 3.0 / 2.0 / 5.0 ]
Paths of length k = 3		1.0 [ 1.0 / 0.0 / 1.0 ]

('d', 'c', 'd') -> 1.0
('d', 'c', 'b') -> 1.0
('a', 'b', 'a') -> 1.0
('a', 'b', 'c', 'd') -> 1.0


Теперь мы наблюдаем один причинный путь a-> b-> c-> d длины три и три дополнительных причинных пути длины два. Все более короткие причинные пути содержатся в этих более длинных причинных путях, как показывает статистика путей выше.

Для анализа сетевых данных с отметками времени крайне важно понимать, какие существуют причинные пути, поскольку только на таких причинных путях узлы могут прямо или косвенно влиять друг на друга. pathpy позволяет анализировать результирующую причинную топологию в данных временных рядов. Более того, в одном из будущих модулей мы увидим, как мы можем использовать сетевые модели более высокого и многоуровневого порядка для анализа причинно-следственных связей и выявления значимых закономерностей в данных временных рядов.

In [27]:
# docstrings
help(pp.TemporalNetwork)

Help on class TemporalNetwork in module pathpy.classes.temporal_network:

class TemporalNetwork(builtins.object)
 |  TemporalNetwork(tedges=None)
 |  
 |  This class represents a sequence of time-stamped edges.
 |   Instances of this class can be used to generate path statistics
 |   based on the time-respecting paths resulting from a given maximum
 |   time difference between consecutive time-stamped edges.
 |  
 |  Methods defined here:
 |  
 |  __init__(self, tedges=None)
 |      Constructor that generates a temporal network instance.
 |      
 |      Parameters
 |      ----------
 |      tedges:
 |          an optional list of directed time-stamped edges from which to construct a
 |          temporal network instance. For the default value None an empty temporal
 |          network will be created.
 |  
 |  __str__(self)
 |      Returns the default string representation of this temporal network instance.
 |  
 |  add_edge(self, source, target, ts, directed=True, timestamp_format='%

### higher-order network models

[source](https://www.pathpy.net/tutorial/higher_order.html)

The class HigherOrderNetwork allows us to generate such higher-order generalisations of network models of paths. The constructor of this class takes a parameter paths, which contains the statistics of observed paths that we want to model. The parameter k allows us to specify the order k of the higher-order model that we want to fit. To understand this better, let us reuse our example from the previous unit:

In [36]:
p = pp.Paths()
p.add_path(('a', 'c', 'd'), frequency=10)
p.add_path(('b', 'c', 'e'), frequency=10)

hon_1 = pp.HigherOrderNetwork(p, k=1)
print(hon_1)

Higher-order network of order k = 1

Nodes:				5
Links:				4
Total weight (subpaths/longest paths):	40.0/0.0



Это создает модель первого порядка наших путей с пятью узлами a, b, c, d и e и четырьмя звеньями (a, c), (b, c), (c, d), (c, e) . Он практически идентичен экземпляру Network, который мы создали в предыдущем модуле с помощью метода Network.from_paths. Действительно, класс HigherOrderNetwork является производным от класса Network, что означает, что все методы, доступные для сетей, также могут применяться к экземплярам сети более высокого порядка. Мы можем, например, использовать те же методы для визуализации сетей более высокого порядка, и мы также можем получить доступ к ребрам таким же образом:

In [37]:
style = { 
    'label_offset': [0,-1], 
    'label_color' : 'black', 
    'width': 800, 
    'height': 250 
}

In [38]:
pp.visualisation.plot(hon_1, **style)

In [39]:
for e in hon_1.edges:
    print(e, hon_1.edges[e]['weight'])

('a', 'c') [10.  0.]
('c', 'd') [10.  0.]
('b', 'c') [10.  0.]
('c', 'e') [10.  0.]


Этот вывод подтверждает, что модель HigherOrderModel с k = 1 идентична нашей сетевой модели. С одним исключением: **веса ребер являются векторами**. Как и в случае с путями, первая запись фиксирует частоту подпути, а вторая запись учитывает появление ребра как самый длинный путь.

Мы можем рассматривать эту сеть как модель первого порядка для путей, где ребра - это пути длины 1. То есть в модели с порядком k = 1 веса ребер фиксируют статистику путей длиной k = 1. Мы можем обобщить эту идею на модели k-го порядка для путей, где узлами являются пути длины k − 1, а веса ребер фиксируют статистику путей длины k. Мы можем сгенерировать такую модель k-го порядка, выполнив преобразование линейного графа на модели с порядком k − 1. То есть ребра в модели порядка k − 1 становятся узлами в модели порядка k. Затем мы рисуем ребра между узлами более высокого порядка всякий раз, когда есть возможный путь длины k в базовой сети. Результатом является k-мерная модель графа Де Брейна для путей. Давайте попробуем это на нашем примере: 

In [42]:
hon_2 = pp.HigherOrderNetwork(p, k=2)
pp.visualisation.plot(hon_2, **style)

for e in hon_2.edges:
    print(e, hon_2.edges[e])

('a,c', 'c,d') {'weight': array([ 0., 10.])}
('b,c', 'c,e') {'weight': array([ 0., 10.])}


Каждое из четырех ребер в модели первого порядка теперь представлено узлом в модели второго порядка. Кроме того, у нас есть два направленных ребра (a − c, c − d) и (b − c, c − e), которые представляют два пути длины два, которые встречаются в наших данных.

Это важно, потому что он фиксирует, в какой степени пути, которые мы наблюдаем в наших данных, отклоняются от того, что мы ожидаем, исходя из сетевой топологии (первого порядка) системы. Рассматривая такую модель первого порядка, все четыре пути a-> c-> d, a-> c-> e, b-> c-> d и b-> c-> e длины два возможны. Если бы края были статистически независимыми, мы бы ожидали, что эти четыре пути будут происходить с одинаковой частотой.

Другой способ выразить это предположение о независимости - рассмотреть модели цепей Маркова для последовательностей узлов, пройденных путем. С этой точки зрения независимо возникающие ребра переводятся в марковский процесс без памяти первого порядка для последовательности узлов. В нашем примере мы ожидаем, что пути a-> c-> d и a-> c-> e возникнут с одинаковой вероятностью, то есть следующие узлы d или e на пути через c не зависят от предыдущего узла a, их вероятности только в зависимости от относительной частоты ребер (c, d) по сравнению с (c, e). В нашем игрушечном примере у нас всего 20 наблюдаемых путей длиной два, поэтому мы ожидаем, что каждый из этих путей будет повторяться в среднем 5 раз.

pathpy может генерировать нулевые модели для путей в пространстве возможных моделей второго порядка. Это позволяет нам сравнить, как наблюдаемая статистика траекторий отклоняется от (марковского) ожидания.

In [43]:
hon_2_null = pp.HigherOrderNetwork(p, k=2, null_model=True)
pp.visualisation.plot(hon_2_null, **style)

for e in hon_2_null.edges:
    print(e, hon_2_null.edges[e])

('a,c', 'c,d') {'weight': array([0., 5.])}
('a,c', 'c,e') {'weight': array([0., 5.])}
('b,c', 'c,d') {'weight': array([0., 5.])}
('b,c', 'c,e') {'weight': array([0., 5.])}


Выходные данные подчеркивают, что пути b-> c-> e и a-> c-> d встречаются в пять раз чаще, чем мы ожидали бы случайным образом, в то время как два других пути встречаются в пять раз меньше, чем ожидалось. Это отклонение от наших ожиданий меняет причинную топологию системы, то есть кто на кого может влиять. В сетевой модели мы неявно предполагаем, что пути транзитивны, т.е. поскольку узел a подключен к узлу c, а узел c подключен к узлу d, мы предполагаем, что существует путь, по которому a может влиять на d через узел c. Модель второго порядка в нашем игрушечном примере показывает, что это предположение о транзитивности вводит в заблуждение, выделяя в наших данных зависимости более высокого порядка, которые приводят к тому, что ни a не может влиять на d, ни b не может влиять на e.

### find optimal higher-order models?

[source](https://www.pathpy.net/tutorial/model_selection.html)

Как мы можем решить, какой порядок мы должны использовать для моделирования данного набора данных? И как мы решаем, существенно ли отклоняется статистика путей от транзитивного, марковского предположения, сделанного в первую очередь моделью первого порядка. Таким образом, нам нужны методы, чтобы решить, когда действительно нужны модели более высокого порядка и какой порядок является оптимальным для моделирования путей.

Более того, модель более высокого порядка с порядком k может фиксировать зависимости только более высокого порядка при одной фиксированной длине корреляции k. Но мы можем столкнуться с данными, которые демонстрируют сразу несколько длин корреляции. Как мы можем объединить модели с несколькими более высокими порядками в модель с несколькими порядками?

В этом модуле мы используем статистический вывод и перспективу машинного обучения, чтобы ответить на эти вопросы. 

In [53]:
p = pp.Paths()
p.add_path('a,c,d', 2)
p.add_path('b,c,e', 2)
print(p)

Total path count: 		4.0 
[Unique / Sub paths / Total]: 	[2.0 / 20.0 / 24.0]
Nodes:				5 
Edges:				4
Max. path length:		2
Avg path length:		2.0 
Paths of length k = 0		0.0 [ 0.0 / 12.0 / 12.0 ]
Paths of length k = 1		0.0 [ 0.0 / 8.0 / 8.0 ]
Paths of length k = 2		4.0 [ 2.0 / 0.0 / 4.0 ]



Как подчеркивалось в предыдущем модуле, в этом примере мы наблюдаем только два из четырех путей длиной два, которые были бы возможны в нулевой модели. Следовательно, это пример статистики пути, которая демонстрирует корреляции, которые требуют модели второго порядка.

Но как мы можем решить это значимым образом? Мы можем использовать статистический вывод по проблеме. Более конкретно, мы будем рассматривать наши сети более высокого порядка как вероятностные генеративные модели для путей в данной топологии сети. Для этого воспользуемся взвешенной сетевой моделью первого порядка, чтобы построить матрицу перехода модели цепи Маркова для путей в сети. Мы просто используем относительные частоты ребер, чтобы пропорционально масштабировать вероятности переходов пл ребрам в модели.

In [46]:
hon_1 = pp.HigherOrderNetwork(p)
pp.visualisation.plot(hon_1)
print(hon_1.transition_matrix())

  (1, 0)	1.0
  (1, 3)	1.0
  (2, 1)	0.5
  (4, 1)	0.5


Эту матрицу перехода можно рассматривать как модель цепи Маркова первого порядка для путей в базовой топологии сети. Этот вероятностный взгляд позволяет нам вычислить вероятность модели первого порядка, учитывая пути, которые мы наблюдали. С помощью pathpy мы можем напрямую вычислить вероятность модели более высокого порядка с учетом экземпляра Paths.

In [48]:
print(hon_1.likelihood(p, log=False))

0.0625


Этот результат особенно легко понять на нашем игрушечном примере. Каждый путь длиной два соответствует двум переходам в матрице переходов нашей модели цепи Маркова. Для каждого из четырех путей длиной два в p первый переход является детерминированным, поскольку узлы a и b указывают только на узел c. Однако, исходя из топологии сети, на втором этапе у нас есть выбор между узлами d и e. Учитывая, что мы видим столько переходов через ребро (c, d), сколько мы видим через ребро (c, e), в модели первого порядка у нас нет причин отдавать предпочтение одному перед другим, поэтому каждому присваивается вероятность 0,5.

Следовательно, для каждого из четырех наблюдаемых путей мы получаем вероятность 1⋅0,5 = 0,5, что дает общую вероятность для четырех (независимых) наблюдений 0,54, равную 0,0625.

Давайте сравним это с вероятностью модели второго порядка для наших путей.

In [49]:
hon_2 = pp.HigherOrderNetwork(p, k=2)
print(hon_2.transition_matrix())
hon_2.likelihood(p, log=False)

  (1, 0)	1.0
  (3, 2)	1.0


1.0

Здесь вероятность принимает максимальное значение 1 просто потому, что все переходы в модели второго порядка детерминированы, то есть мы умножаем 1⋅1 четыре раза.

Давайте теперь посмотрим на нулевую модель второго порядка, которая на самом деле является моделью первого порядка, представленной в пространстве второго порядка. Таким образом, мы должны ожидать такой же вероятности, что и модель первого порядка.

In [50]:
hon_2_null = pp.HigherOrderNetwork(p, k=2, null_model=True)
pp.visualisation.plot(hon_2_null)
print(hon_2.transition_matrix())
hon_2_null.likelihood(p, log=False)

  (1, 0)	1.0
  (3, 2)	1.0


0.0625

Ясно, что нуль второго порядка должен иметь такую же вероятность, что и модель первого порядка. Это также показывает способ проверки гипотез о наличии корреляций более высокого порядка в путях. Мы можем использовать тест отношения правдоподобия, чтобы сравнить вероятность нулевой гипотезы (представления второго порядка модели первого порядка) с вероятностью альтернативной гипотезы (подобранная модель второго порядка).

Но что мы узнаем из того факта, что вероятность модели возрастает по мере того, как мы увеличиваем порядок модели. Само собой не очень много. Модели более высокого порядка сложнее, чем модели первого порядка, то есть при подборе их матрицы перехода мы фактически подбираем больше параметров к данным. Таким образом, мы можем ожидать, что такая более сложная модель лучше объясняет наши (пути) данные.

Мы должны напомнить себе о бритве Оккама, которая гласит, что **мы должны отдавать предпочтение моделям, которые делают меньше предположений**. Таким образом, при сравнении вероятностей моделей мы должны учитывать дополнительную сложность (или степени свободы) модели более высокого порядка по сравнению с нулевой гипотезой модели первого порядка.

В конкретном случае, который мы рассматриваем, мы можем применить теорему Уилка, чтобы вывести аналитическое выражение для p-значения нулевой гипотезы об отсутствии зависимостей второго порядка (т.е. модели первого порядка достаточно для объяснения наблюдаемых путей), по сравнению с альтернативной гипотезой о необходимости модели второго порядка. Полная информация об этом подходе к выбору модели доступна в [этом документе KDD](https://dl.acm.org/doi/10.1145/3097983.3098145).

Давайте применим это, чтобы проверить гипотезу о наличии существенных зависимостей второго порядка в нашем игрушечном примере. Тест состоит из трех этапов:

- вычислить разность d между параметрами (или степенями свободы) модели второго и первого порядка.
- рассчитать тестовую статистику для теста отношения правдоподобия.
- использовать статистику теста и разность степеней свободы, чтобы вычислить p-значение для нулевой гипотезы

Хотя мы опускаем математические детали, это можно сделать с помощью нескольких строк кода Python:

In [51]:
from scipy.stats import chi2

d = hon_2.degrees_of_freedom() - hon_1.degrees_of_freedom()
x = - 2 * (hon_1.likelihood(p, log=True) - hon_2.likelihood(p, log=True))
p = 1 - chi2.cdf(x, d)

print('p-value of null hypothesis (first-order model) is {0}'.format(p))

p-value of null hypothesis (first-order model) is 0.018531677751199016


Мы находим p-значение 0,019. Это интуитивно понятно, поскольку мы наблюдали только четыре пути, что вряд ли достаточно, чтобы иметь веские доказательства против модели первого порядка. Посмотрим, что произойдет, если мы будем чаще следовать одним и тем же путям.

In [ ]:
p = pp.Paths()
p.add_path('a,c,d', 2)
p.add_path('b,c,e', 2)

In [54]:
p *= 10
x = - 2 * (hon_1.likelihood(p, log=True) - hon_2.likelihood(p, log=True))
p = 1 - chi2.cdf(x, d)

print('p-value of null hypothesis (first-order model) is {0}'.format(p))

p-value of null hypothesis (first-order model) is 9.581224702515101e-14


Если бы мы наблюдали каждый из двух путей в десять раз чаще, у нас были бы более веские доказательства, говорящие против нулевой гипотезы и, следовательно, в пользу модели второго порядка. Если бы мы еще больше увеличили количество наблюдений за траекториями, p-значение еще больше уменьшилось бы.

К сожалению, приведенный выше пример слишком прост во многих отношениях: во-первых, он содержит только пути длиной ровно два, что оправдывает модель второго порядка. Но реальные данные более сложны, поскольку у нас есть наблюдения за путями на разных длинах одновременно. Такие данные, вероятно, одновременно будут иметь несколько длин корреляции.

Что еще более важно, в реальных данных выбор модели, к сожалению, не будет работать, как описано выше. Фактически, мы обманули, потому что мы не можем, как правило, напрямую сравнивать вероятности моделей разного порядка. Следующий пример подчеркивает эту проблему:

In [55]:
path = ('a','b','c','d','e','c','b','a','c','d','e','c','e','d','c','a')

p = pp.Paths()
p.add_path(path)
pp.visualisation.plot(pp.Network.from_paths(p))

hon_1 = pp.HigherOrderNetwork(p, k=1)
hon_2 = pp.HigherOrderNetwork(p, k=2, null_model=True)
hon_5 = pp.HigherOrderNetwork(p, k=5, null_model=True)

print(hon_1.likelihood(p, log=False))
print(hon_2.likelihood(p, log=False))
print(hon_5.likelihood(p, log=False))

1.755829903978052e-06
3.511659807956104e-06
2.633744855967078e-05


Разве вероятности этих трех моделей не должны быть одинаковыми? Это не так, и это серьезная проблема, когда у нас есть данные, состоящие из большого количества коротких путей: с точки зрения количества переходов, которые входят в расчет вероятности, модель порядка k отбрасывает первые k узлов на каждом пути. То есть модель второго порядка может учитывать только все обходы ребер на пути, кроме первого. Это означает, что - в общем случае - мы фактически сравниваем вероятности, вычисленные для разных выборок, что неверно. Выделим это, подсчитав количество переходов, которые входят в расчет вероятности:

In [56]:
print('Path consists of {0} nodes'.format(len(path)))
print('first-order model  = ', str(len(hon_1.path_to_higher_order_nodes(path)[1:])))
print('second-order model = ', str(len(hon_2.path_to_higher_order_nodes(path)[1:])))
print('fifth-order model  = ', str(len(hon_5.path_to_higher_order_nodes(path)[1:])))

Path consists of 16 nodes
first-order model  =  15
second-order model =  14
fifth-order model  =  11


Чтобы исправить указанные выше проблемы, нам нужна вероятностная генеративная модель, которая может работать с большими наборами (коротких) путей в сети. Ключевая идея состоит в том, чтобы объединить несколько сетевых моделей более высокого порядка в единую многоуровневую модель нескольких порядков. Чтобы рассчитать вероятность такой модели, мы можем использовать все слои, что позволяет избежать проблемы, связанной с отбрасыванием префиксов путей. Для каждого пути мы начинаем вычисление со слоя нулевого порядка, который учитывает относительные вероятности узлов. Затем мы используем этот модельный слой для расчета вероятности наблюдения первого узла на пути. Затем для следующего перехода ко второму шагу мы используем модель первого порядка. Следующий переход рассчитывается в модели второго порядка и так далее, пока мы не достигнем максимального порядка нашей модели нескольких порядков. На этом этапе мы можем транзитивно вычислить вероятность на основе оставшихся переходов пути.

Математические детали метода описаны в этой статье KDD. Но перейдем к практике. pathpy может напрямую генерировать, визуализировать и анализировать сетевые модели с несколькими порядками. Давайте попробуем это на нашем примере:

In [57]:
mog = pp.MultiOrderModel(p, max_order=2)
print(mog)

2021-01-26 21:54:04 [Severity.INFO]	Generating 0-th order layer ...
2021-01-26 21:54:04 [Severity.INFO]	Generating 1-th order layer ...
2021-01-26 21:54:04 [Severity.INFO]	Generating 2-th order layer ...
2021-01-26 21:54:04 [Severity.INFO]	finished.
Multi-order model (max. order = 2, DoF (paths/ngrams) = 38 / 124)
Layer k = 0 	 6 nodes, 5 links, 16.0 paths, DoF (paths/ngrams) = 4 / 4 
Layer k = 1 	 5 nodes, 12 links, 15.0 paths, DoF (paths/ngrams) = 7 / 20 
Layer k = 2 	 12 nodes, 12 links, 14.0 paths, DoF (paths/ngrams) = 27 / 100 



Теперь мы можем использовать функцию правдоподобия класса MultiOrderModel, чтобы повторить наш тест отношения правдоподобия. Вместо того, чтобы создавать несколько экземпляров MultiOrderModel для разных гипотез, мы можем напрямую вычислять вероятности на основе разных слоев модели в одном экземпляре MultiOrderModel.

In [58]:
mog = pp.MultiOrderModel(p, max_order=2)

d = mog.degrees_of_freedom(max_order=2) - mog.degrees_of_freedom(max_order=1)
x = - 2 * (mog.likelihood(p, log=True, max_order=1) 
    - mog.likelihood(p, log=True, max_order=2))
p = 1 - chi2.cdf(x, d)

print('p value of null hypothesis that data has maximum order 1 = {0}'.format(p))

2021-01-26 21:55:28 [Severity.INFO]	Generating 0-th order layer ...
2021-01-26 21:55:28 [Severity.INFO]	Generating 1-th order layer ...
2021-01-26 21:55:28 [Severity.INFO]	Generating 2-th order layer ...
2021-01-26 21:55:28 [Severity.INFO]	finished.
p value of null hypothesis that data has maximum order 1 = 0.7195815409015258


Мы находим убедительные доказательства против нулевой гипотезы о том, что пути можно объяснить с помощью сетевой модели первого порядка. Фактически мы получаем другое значение p, так как мы также учитываем модель нулевого порядка, т.е. мы учитываем относительные частоты, с которыми узлы встречаются в начале пути.

Вместо того, чтобы выполнять проверку правдоподобия самостоятельно, мы можем просто вызвать метод MultiOrderModel.estimate_order. он вернет максимальный порядок среди всех своих слоев, для которого проверка отношения правдоподобия отклоняет нулевую гипотезу.

In [59]:
mog.estimate_order()

2021-01-26 21:57:08 [Severity.INFO]	Likelihood ratio test for K_opt = 2, x = 22.346254782758862
2021-01-26 21:57:08 [Severity.INFO]	Likelihood ratio test, d_1-d_0 = 27
2021-01-26 21:57:08 [Severity.INFO]	Likelihood ratio test, p = 0.7195815409015258


1

Теперь мы проверим, действительно ли работает этот подход для изучения оптимального представления данных пути. Для этого давайте сгенерируем статистику путей, которая соответствует тому, что мы ожидаем, на основе сетевой модели первого порядка, и проверим, дает ли оценка порядка правильный результат.

In [60]:
random_paths = pp.Paths()
random_paths.add_path('a,c,d', 5)
random_paths.add_path('a,c,e', 5)
random_paths.add_path('b,c,e', 5)
random_paths.add_path('b,c,d', 5)

mog = pp.MultiOrderModel(random_paths, max_order=2)
print('Optimal order = ', mog.estimate_order(random_paths))

2021-01-26 21:58:01 [Severity.INFO]	Generating 0-th order layer ...
2021-01-26 21:58:01 [Severity.INFO]	Generating 1-th order layer ...
2021-01-26 21:58:01 [Severity.INFO]	Generating 2-th order layer ...
2021-01-26 21:58:01 [Severity.INFO]	finished.
2021-01-26 21:58:01 [Severity.INFO]	Likelihood ratio test for K_opt = 2, x = -0.0
2021-01-26 21:58:01 [Severity.INFO]	Likelihood ratio test, d_1-d_0 = 2
2021-01-26 21:58:01 [Severity.INFO]	Likelihood ratio test, p = 1.0
Optimal order =  1


В этом примере мы не находим доказательств против модели первого порядка, поскольку все транзитивные пути происходят с точно той частотой, которую мы ожидали бы случайным образом! Следовательно, в этом случае нам не нужны модели более высокого порядка, чтобы понять причинную топологию системы, которая фиксирует, какие узлы могут прямо или косвенно влиять друг на друга с течением времени.